In [104]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [105]:
dt_hoje = '2024-08-08'

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funções

In [153]:
def drop_unnamed_cols(df):
    cols = df.columns
    cols_unnamed = [col for col in cols if str(col).startswith('Unnamed')]

    columns_to_drop = df[cols_unnamed].dropna(axis=1, how='all').columns

    return df.drop(columns=columns_to_drop)



def drop_null_cols(df):
    return df.dropna(axis=1, how='all')



def read_all_tables(dirname):
    all_dfs = {}

    list_files = os.listdir(dirname)

    for file_ in list_files:
        if file_.endswith(".csv"):
            file_name = os.path.splitext(file_)[0]
            df = pd.read_csv(os.path.join(dirname, file_))
            df_cleanned = drop_unnamed_cols(df)
            df_cleanned = drop_null_cols(df_cleanned)
            all_dfs[file_name] = df_cleanned

            print(file_)

    return all_dfs



def print_keys_with_column(data_dict, column_name):
    for key, df in data_dict.items():
        if column_name in df.columns:
            print(key)



def print_keys_with_column_like(data_dict, column_name_like):
    for key, df in data_dict.items():
        matching_columns = [col for col in df.columns if column_name_like in col]
        if matching_columns:
            print(key)



def search_tables_use_column(ls_data_dict, type_search, column_name):
    if type_search == 'like':
        for dict_data in ls_data_dict:
            print_keys_with_column_like(dict_data, column_name)
    elif type_search == 'exact':
        for dict_data in ls_data_dict:
            print_keys_with_column(dict_data, column_name)
    else:
        raise ValueError("Tipo de busca não suportado. Use 'like' ou 'exact'.")



def plot_qtd_stack_bar(df, x, y, group, figsize=(20,10), colors=None):
    # Se a group for nulo o gráfico de barras pode ver utilizado sem criar grupos dentro das barras
    if group:
        agg_tips = df.groupby([x, group])[y].count().unstack().fillna(0)
    else:
        agg_tips = df.groupby([x])[y].count().fillna(0).to_frame()

    fig, ax = plt.subplots(figsize=figsize)

    # colors = ['#008080', '#DAA520','#8A2BE2','#0000FF']
    if colors:
        pass
    else:
        colors = [None for i in range(agg_tips.shape[1])]

    bottom = np.zeros(len(agg_tips))

    lst_index = [str(i) for i in agg_tips.index]

    for i, col in enumerate(agg_tips.columns):
        ax.bar(
          lst_index, agg_tips[col], bottom=bottom, label=col, edgecolor = "black", color=colors[i]) #, color=colors[i]
        bottom += np.array(agg_tips[col])

    # Coloca o label de qtd de observações acima das barras.
    totals = agg_tips.sum(axis=1)
    lst_index_total = [str(i) for i in totals.index]
    y_offset = 5
    for i, total in enumerate(totals):
        ax.text(lst_index_total[i], total + y_offset, round(total), ha='center', weight='bold')

    # Se group é nulo não colocamos o label de qtd de observações dentro da barra
    if group:
        # Let's put the annotations inside the bars themselves by using a
        # negative offset.
        y_offset = -100
        # For each patch (basically each rectangle within the bar), add a label.
        for bar in ax.patches:
            ax.text(
                # Put the text in the middle of each bar. get_x returns the start
                # so we add half the width to get to the middle.
                bar.get_x() + bar.get_width() / 2,
                # Vertically, add the height of the bar to the start of the bar,
                # along with the offset.
                bar.get_y() + bar.get_height() / 2, #bar.get_height() + bar.get_y() + y_offset,
                # This is actual value we'll show.
                round(bar.get_height()),
                # Center the labels and style them a bit.
                ha='center',
                color='w',
                weight='bold',
                size=8
            )

        ax.set_title(f'Qtd de {y} por {group}')
        ax.legend()
        plt.xticks(rotation=90)

    else:
        ax.set_title(f'Qtd de {y}')
        ax.legend()
        plt.xticks(rotation=90)



def plot_porcentagem_stack_bar(df, x, y, group, figsize=(20,10)):
    agg_tips = df.groupby([x, group])[y].count().unstack().fillna(0)
    totals = agg_tips.sum(axis=1)
    agg_tips = agg_tips.div(totals, axis='index').mul(100)

    fig, ax = plt.subplots(figsize=figsize)

    # colors = ['#008080', '#DAA520','#8A2BE2','#0000FF']
    bottom = np.zeros(len(agg_tips))

    lst_index = [str(i) for i in agg_tips.index]

    for i, col in enumerate(agg_tips.columns):
        ax.bar(
          lst_index, agg_tips[col], bottom=bottom, label=col, edgecolor = "black") # , color=colors[i]

        bottom += np.array(agg_tips[col])

    # For each patch (basically each rectangle within the bar), add a label.
    for bar in ax.patches:
        ax.text(
            # Put the text in the middle of each bar. get_x returns the start
            # so we add half the width to get to the middle.
            bar.get_x() + bar.get_width() / 2,
            # Vertically, add the height of the bar to the start of the bar,
            # along with the offset.
            bar.get_y() + bar.get_height() / 2, #bar.get_height() + bar.get_y() + y_offset,
            # This is actual value we'll show.
            round(bar.get_height()),
            # Center the labels and style them a bit.
            ha='center',
            color='w',
            weight='bold',
            size=8
        )

    ax.set_title(f'Porcentagem de matriculas por {group}')
    ax.legend()
    plt.xticks(rotation=90)


def cleanning_cols_dates(df):
    df_ = df.copy()

    lst_cols_dates = [col for col in df_.columns if col.startswith('Data')]
    print("Lista de colunas de datas:", lst_cols_dates)
    for col in lst_cols_dates:
        try:
            df_[col] = pd.to_datetime(df_[col])
        except:
            print(f'Essa coluna "{col}" não pode ser convertida para data')

    return df_


def coalesce_multiple_columns(df, lst_cols_to_coalesce, new_col, drop_cols=False):
    df_ = df.copy()
    # Nova coluna
    df_[new_col] = df_[lst_cols_to_coalesce].bfill(axis=1).iloc[:, 0]

    if drop_cols:
        df_ = df_.drop(columns=lst_cols_to_coalesce)

    return df_

# Lendo todas as tabelas

In [108]:
TbOutras = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/Outras tabelas/")
TbAbatimento = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbAbatimento/Originais anonimizados")
TbAlunos = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbAluno/Originais anonimizados")
TbCampoDinamico = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbCampoDinamico/Originais anonimizados")
TbCaptacao = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbCaptacao/Originais anonimizados")
TbDiario = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbDiario/Originais anonimizados")
TbFase = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbFase/Originais anonimizados")
TbHistorico = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbHistorico/Originais anonimizados")
TbMeta = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbMeta")
TbProfessor = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbProfessor/Originais anonimizados")
TbResponsavel = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbResponsavel/Originais anonimizados")
TbSerie = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbSerie/Originais anonimizados")
TbSituacaoAlunoDisciplina = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbSituacaoAlunoDisciplina/Originais anonimizados")
TbTurma = read_all_tables("/content/drive/MyDrive/Pos tech/datathon/csv_output/Tabelas/TbTurma/Originais anonimizados")
lst_todos_dados = [TbOutras, TbAbatimento, TbAlunos, TbCampoDinamico, TbCaptacao, TbDiario, TbFase, TbHistorico, TbMeta, TbProfessor, TbResponsavel, TbSerie, TbSituacaoAlunoDisciplina, TbTurma]

DictTodasTabelas = (TbOutras | TbAbatimento | TbAlunos | TbCampoDinamico | TbCaptacao | TbDiario | TbFase | TbHistorico | TbMeta | TbProfessor | TbResponsavel | TbSerie | TbSituacaoAlunoDisciplina | TbTurma)

TbPais.csv
TbMunicipio.csv
TbDisciplina.csv
TbCentroResultado.csv
TbPeriodo.csv
TbMotivoInativacao.csv
TbFormaIngresso.csv
TbFreqQuadroHorario.csv
TbCursoFases.csv
TbTipoOcorrencia.csv
TbGradeCurricular.csv
TbAbatimentoTipo.csv
TbAbatimento.csv
TbAlunoRotinaEducacaoInfantil.csv
TbAlunoTurmaHistorico.csv
TbAlunoProprioResponsavel.csv
TbAlunoObs.csv
TbAluno.csv
TbAlunoTurma.csv
TbCampoDinamicoConjuntoElemento.csv
TbCampoDinamico.csv
TbCampoDinamicoConjunto.csv
TbCaptacaoCursoInteresse.csv
TbCaptacaoMotivoDesistencia.csv
TbCaptacaoSituacaoLead.csv
TbCaptacaoOrigemLead.csv
TbDiarioFrequencia.csv
TbDiarioAluno.csv
TbDiario.csv
TbDiarioAula.csv
TbFaseNotaAluno.csv
TbFaseNota.csv
TbFaseNotaDisciplinaTurma.csv
TbFaseNotaOrigemDestino.csv
TbHistorico.csv
TbHistoricoNotas.csv
TbMeta.csv
TbMetaFaseNota.csv
TbMetaFaseNotaAluno.csv
TbMetaSituacaoAlunoDisciplina.csv
TbTipoMeta.csv
TbMetaConceito.csv
TbProfessorDisciplina.csv
TbProfessorHorario.csv
TbProfessor.csv
TbTipoVinculoAlunoResponsavel.csv
Tb

In [109]:
# Dataframes importantes
alunos_turmas_df = TbAlunos['TbAlunoTurma']
alunos_df = TbAlunos['TbAluno']
alunos_turmas_hist_df = TbAlunos['TbAlunoTurmaHistorico']
disciplinas_df = TbSerie['TbSerie']
turmas_df = TbTurma['TbTurma']
situacao_aluno_turma_df = TbTurma['TbSituacaoAlunoTurma']
motivos_inat_df = TbOutras['TbMotivoInativacao']

In [126]:
TbOutras['TbFreqQuadroHorario'][TbOutras['TbFreqQuadroHorario']['IdTurma'] == 235]

,IdTurmaHorario,IdTurma,DiaSemana,Aula,IdDisciplina
0,1,235,2,T6,1
1,2,235,2,T7,1
2,3,235,2,T8,1
3,4,235,5,T6,4
4,5,235,5,T7,4
5,6,235,5,T8,4
6,7,235,6,T3,16
7,8,235,6,T4,16
708,716,235,5,T1,18
709,717,235,5,T2,18


In [120]:
TbDiario.keys()

dict_keys(['TbDiarioFrequencia', 'TbDiarioAluno', 'TbDiario', 'TbDiarioAula'])

# Dados de presença e faltas

In [121]:
diario_freq_df = TbDiario['TbDiarioFrequencia'] # Tabela com os frequências, uma linha por IdDiarioFrequencia
diario_df = TbDiario['TbDiario'] # Tabela com os dados do diário, uma linha por IdDiario
diario_aula_df = TbDiario['TbDiarioAula'] # Tabela com os dados da aula, uma linha por IdDiarioAula
diario_aluno_df = TbDiario['TbDiarioAluno'] # Tabela com os dados do aluno, uma linha por IdDiarioAluno

In [122]:
# Cruzandos as tabela para montar um analítico de frequências
print("Shape de diario_freq_df:", diario_freq_df.shape)
analitico_freq_df = diario_freq_df.merge(diario_aula_df, on='IdDiarioAula', how='left')
analitico_freq_df = analitico_freq_df.merge(diario_df, on='IdDiario', how='left')
analitico_freq_df = analitico_freq_df.merge(diario_aluno_df, on=['IdDiario','IdAluno'], how='left')

print("Shape de analitico_freq_df:", analitico_freq_df.shape)

Shape de diario_freq_df: (313160, 4)
Shape de analitico_freq_df: (313160, 26)


In [123]:
# Limpeza de colunas
analitico_freq_df = cleanning_cols_dates(analitico_freq_df) # Limpeza de colunas de datas

Lista de colunas de datas: ['DataAula', 'DataHoraLimiteChamadaOnline', 'DataLimiteDigitacao', 'DataBloqueioDigitacaoAula', 'DataInicial', 'DataFinal']
Essa coluna "DataHoraLimiteChamadaOnline" não pode ser convertida para data


<ipython-input-107-1c7778294fc3>:175: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_[col] = pd.to_datetime(df_[col])


In [124]:
# Adicionando colunas
analitico_freq_df['AnoMesAula'] = analitico_freq_df['DataAula'].dt.strftime('%Y%m').astype(int)

In [198]:
# Consolidando frequências por ano, mês, diário e disciplina
frequencia_aluno_turma_df = analitico_freq_df.groupby(by=[
    'IdAluno',
    'IdTurma',
    # 'IdDiario',
    'IdDisciplina',
    'AnoMesAula',
    'StPresencaFalta'
], as_index=False).agg(
    QtdAulas=('IdDiarioFrequencia', 'nunique'),
    # MaxQtdeMinimaAulas=('QtdeMinimaAulas', 'max'),
    # MinQtdeMinimaAulas=('QtdeMinimaAulas', 'min'),
    # MaxQtdeMaximaAulas=('QtdeMaximaAulas', 'max'),
    # MinQtdeMaximaAulas=('QtdeMaximaAulas', 'min')
)

# # Removendo colunas
# if frequencia_aluno_turma_df[
#     (frequencia_aluno_turma_df['MaxQtdeMinimaAulas'] != frequencia_aluno_turma_df['MinQtdeMinimaAulas']) |
#     (frequencia_aluno_turma_df['MaxQtdeMaximaAulas'] != frequencia_aluno_turma_df['MinQtdeMaximaAulas'])
# ].shape[0] == 0:
#     frequencia_aluno_turma_df = frequencia_aluno_turma_df.drop(columns=['MaxQtdeMinimaAulas','MaxQtdeMaximaAulas'])
#     frequencia_aluno_turma_df = frequencia_aluno_turma_df.rename(columns={'MinQtdeMinimaAulas': 'QtdeMinimaAulas', 'MinQtdeMaximaAulas': 'QtdeMaximaAulas'})

In [199]:
# Pivoteando a coluna de StPresencaFalta
frequencia_aluno_turma_df = frequencia_aluno_turma_df.pivot(
    index=[
        'IdAluno',
        'IdTurma',
        # 'IdDiario',
        'IdDisciplina',
        'AnoMesAula'],
    columns='StPresencaFalta',
    values=[
        'QtdAulas',
        # 'QtdeMinimaAulas',
        # 'QtdeMaximaAulas'
        ]
).reset_index()

# # Tratando colunas
# frequencia_aluno_turma_df = coalesce_multiple_columns(df=frequencia_aluno_turma_df, lst_cols_to_coalesce=[('QtdeMinimaAulas', 'P'),('QtdeMinimaAulas', 'F'),('QtdeMinimaAulas', 'J')], new_col=('QtdeMinimaAulas', ''), drop_cols=True)
# frequencia_aluno_turma_df = coalesce_multiple_columns(df=frequencia_aluno_turma_df, lst_cols_to_coalesce=[('QtdeMaximaAulas', 'P'),('QtdeMaximaAulas', 'F'),('QtdeMaximaAulas', 'J')], new_col=('QtdeMaximaAulas', ''), drop_cols=True)

frequencia_aluno_turma_df.columns = [
    'IdAluno',
    'IdTurma',
    # 'IdDiario',
    'IdDisciplina',
    'AnoMesAula',
    'QtdAulasFalta',
    'QtdAulasJustificada',
    'QtdAulasPresente',
    # 'QtdeMinimaAulas',
    # 'QtdeMaximaAulas'
]

In [200]:
# Tratando nulos
frequencia_aluno_turma_df = frequencia_aluno_turma_df.fillna({'QtdAulasFalta': 0, 'QtdAulasJustificada': 0, 'QtdAulasPresente': 0})

## Criando novas colunas
frequencia_aluno_turma_df['QtdAulasTotal'] = frequencia_aluno_turma_df['QtdAulasFalta'] + frequencia_aluno_turma_df['QtdAulasJustificada'] + frequencia_aluno_turma_df['QtdAulasPresente']

# Criando Features acumuladas
frequencia_aluno_turma_df = frequencia_aluno_turma_df.sort_values(by=['IdAluno','IdTurma','IdDisciplina','AnoMesAula'], ascending=[True,True,True,True])
frequencia_aluno_turma_df['QtdAulasFaltaAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasFalta'].cumsum()
frequencia_aluno_turma_df['QtdAulasJustificadaAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasJustificada'].cumsum()
frequencia_aluno_turma_df['QtdAulasPresenteAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasPresente'].cumsum()
frequencia_aluno_turma_df['QtdAulasTotalAcum'] = frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['QtdAulasTotal'].cumsum()

# Criando features de %
frequencia_aluno_turma_df['PercFalta'] = frequencia_aluno_turma_df['QtdAulasFalta'] / frequencia_aluno_turma_df['QtdAulasTotal']
frequencia_aluno_turma_df['PercJustificada'] = frequencia_aluno_turma_df['QtdAulasJustificada'] / frequencia_aluno_turma_df['QtdAulasTotal']
frequencia_aluno_turma_df['PercPresente'] = frequencia_aluno_turma_df['QtdAulasPresente'] / frequencia_aluno_turma_df['QtdAulasTotal']

frequencia_aluno_turma_df['PercFaltaAcum'] = frequencia_aluno_turma_df['QtdAulasFaltaAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']
frequencia_aluno_turma_df['PercJustificadaAcum'] = frequencia_aluno_turma_df['QtdAulasJustificadaAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']
frequencia_aluno_turma_df['PercPresenteAcum'] = frequencia_aluno_turma_df['QtdAulasPresenteAcum'] / frequencia_aluno_turma_df['QtdAulasTotalAcum']

In [201]:

frequencia_aluno_turma_df

,IdAluno,IdTurma,IdDisciplina,AnoMesAula,QtdAulasFalta,QtdAulasJustificada,QtdAulasPresente,QtdAulasTotal,QtdAulasFaltaAcum,QtdAulasJustificadaAcum,QtdAulasPresenteAcum,QtdAulasTotalAcum,PercFalta,PercJustificada,PercPresente,PercFaltaAcum,PercJustificadaAcum,PercPresenteAcum
0,2,466,4,202305,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.00,0.00,1.00,0.00,0.00,1.00
1,2,466,19,202305,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.00,0.00,1.00,0.00,0.00,1.00
2,3,106,1,202107,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.00,0.00,1.00,0.00,0.00,1.00
3,3,106,1,202108,0.0,0.0,4.0,4.0,0.0,0.0,5.0,5.0,0.00,0.00,1.00,0.00,0.00,1.00
4,3,106,1,202109,0.0,0.0,5.0,5.0,0.0,0.0,10.0,10.0,0.00,0.00,1.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104749,2238,668,4,202406,0.0,0.0,4.0,4.0,0.0,0.0,4.0,4.0,0.00,0.00,1.00,0.00,0.00,1.00
104750,2238,668,16,202406,0.0,1.0,3.0,4.0,0.0,1.0,3.0,4.0,0.00,0.25,0.75,0.00,0.25,0.75
104751,2241,699,1,202406,1.0,0.0,3.0,4.0,1.0,0.0,3.0,4.0,0.25,0.00,0.75,0.25,0.00,0.75
104752,2241,699,4,202406,0.0,0.0,4.0,4.0,0.0,0.0,4.0,4.0,0.00,0.00,1.00,0.00,0.00,1.00


In [177]:
# Aluno com mais de um IdDiario por IdDisciplina
frequencia_aluno_turma_df.groupby(by=['IdAluno','IdTurma','IdDisciplina'])['IdDiario'].nunique().sort_values(ascending=False)

IdAluno  IdTurma  IdDisciplina
781      582      1               4
755      603      25              4
346      501      1               4
                  4               4
                  22              4
                                 ..
630      120      16              1
         130      1               1
                  4               1
                  16              1
2241     699      16              1
Name: IdDiario, Length: 19298, dtype: int64

In [202]:
frequencia_aluno_turma_df[(frequencia_aluno_turma_df['IdAluno'] == 781) & (frequencia_aluno_turma_df['IdDisciplina'] == 1)]

,IdAluno,IdTurma,IdDisciplina,AnoMesAula,QtdAulasFalta,QtdAulasJustificada,QtdAulasPresente,QtdAulasTotal,QtdAulasFaltaAcum,QtdAulasJustificadaAcum,QtdAulasPresenteAcum,QtdAulasTotalAcum,PercFalta,PercJustificada,PercPresente,PercFaltaAcum,PercJustificadaAcum,PercPresenteAcum
52557,781,104,1,202108,2.0,0.0,2.0,4.0,2.0,0.0,2.0,4.0,0.500000,0.0,0.500000,0.500000,0.0,0.500000
52558,781,104,1,202109,0.0,0.0,5.0,5.0,2.0,0.0,7.0,9.0,0.000000,0.0,1.000000,0.222222,0.0,0.777778
52559,781,104,1,202110,0.0,0.0,3.0,3.0,2.0,0.0,10.0,12.0,0.000000,0.0,1.000000,0.166667,0.0,0.833333
52560,781,104,1,202111,0.0,0.0,4.0,4.0,2.0,0.0,14.0,16.0,0.000000,0.0,1.000000,0.125000,0.0,0.875000
52588,781,419,1,202202,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.000000,0.0,1.000000,0.000000,0.0,1.000000
52589,781,419,1,202203,0.0,0.0,5.0,5.0,0.0,0.0,6.0,6.0,0.000000,0.0,1.000000,0.000000,0.0,1.000000
52590,781,419,1,202204,1.0,0.0,3.0,4.0,1.0,0.0,9.0,10.0,0.250000,0.0,0.750000,0.100000,0.0,0.900000
52591,781,419,1,202205,2.0,0.0,2.0,4.0,3.0,0.0,11.0,14.0,0.500000,0.0,0.500000,0.214286,0.0,0.785714
52592,781,419,1,202206,2.0,0.0,3.0,5.0,5.0,0.0,14.0,19.0,0.400000,0.0,0.600000,0.263158,0.0,0.736842
52593,781,419,1,202207,0.0,0.0,1.0,1.0,5.0,0.0,15.0,20.0,0.000000,0.0,1.000000,0.250000,0.0,0.750000


In [179]:
frequencia_aluno_turma_df[(frequencia_aluno_turma_df['IdAluno'] == 781) & (frequencia_aluno_turma_df['IdDisciplina'] == 1)]

,IdAluno,IdTurma,IdDiario,IdDisciplina,AnoMesAula,QtdAulasFalta,QtdAulasJustificada,QtdAulasPresente,QtdeMinimaAulas,QtdeMaximaAulas,...,QtdAulasFaltaAcum,QtdAulasJustificadaAcum,QtdAulasPresenteAcum,QtdAulasTotalAcum,PercFalta,PercJustificada,PercPresente,PercFaltaAcum,PercJustificadaAcum,PercPresenteAcum
54070,781,104,147,1,202108,2.0,0.0,2.0,30.0,35.0,...,2.0,0.0,2.0,4.0,0.500000,0.0,0.500000,0.500000,0.0,0.500000
54071,781,104,147,1,202109,0.0,0.0,5.0,30.0,35.0,...,2.0,0.0,7.0,9.0,0.000000,0.0,1.000000,0.222222,0.0,0.777778
54072,781,104,147,1,202110,0.0,0.0,3.0,30.0,35.0,...,2.0,0.0,10.0,12.0,0.000000,0.0,1.000000,0.166667,0.0,0.833333
54073,781,104,147,1,202111,0.0,0.0,4.0,30.0,35.0,...,2.0,0.0,14.0,16.0,0.000000,0.0,1.000000,0.125000,0.0,0.875000
54100,781,419,428,1,202202,0.0,0.0,1.0,20.0,30.0,...,0.0,0.0,1.0,1.0,0.000000,0.0,1.000000,0.000000,0.0,1.000000
54101,781,419,428,1,202203,0.0,0.0,5.0,20.0,30.0,...,0.0,0.0,6.0,6.0,0.000000,0.0,1.000000,0.000000,0.0,1.000000
54102,781,419,428,1,202204,1.0,0.0,3.0,20.0,30.0,...,1.0,0.0,9.0,10.0,0.250000,0.0,0.750000,0.100000,0.0,0.900000
54103,781,419,428,1,202205,2.0,0.0,2.0,20.0,30.0,...,3.0,0.0,11.0,14.0,0.500000,0.0,0.500000,0.214286,0.0,0.785714
54104,781,419,428,1,202206,2.0,0.0,3.0,20.0,30.0,...,5.0,0.0,14.0,19.0,0.400000,0.0,0.600000,0.263158,0.0,0.736842
54114,781,419,744,1,202207,0.0,0.0,1.0,20.0,30.0,...,0.0,0.0,1.0,1.0,0.000000,0.0,1.000000,0.000000,0.0,1.000000
